In [17]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, StopWordsRemover, HashingTF, IDF, ChiSqSelector
from pyspark.ml.feature import Tokenizer, Normalizer, StringIndexer
from pyspark.sql.functions import col, lower

from pyspark.ml import Pipeline
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from pyspark.mllib.util import MLUtils

In [3]:
partial_dataset = "/user/dic24_shared/amazon-reviews/full/reviews_devset.json"
full_dataset = "/user/dic24_shared/amazon-reviews/full/reviewscombined.json"


In [4]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("TFIDF_ChiSq_Selection") \
    .getOrCreate()

try:
    df = spark.read.json(partial_dataset)
    print("File read successfully.")
    
    # Show the schema and some data
    df.printSchema()
    df.show(5)
    
except Exception as e:
    print(f"Error: {e}")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
24/05/24 14:54:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/24 14:54:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/24 14:54:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/24 14:54:27 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/05/24 14:54:27 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/05/24 14:54:2

File read successfully.
root
 |-- asin: string (nullable = true)
 |-- category: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)

+----------+--------------------+-------+-------+--------------------+-----------+--------------+--------------------+-------------------+--------------+
|      asin|            category|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|            summary|unixReviewTime|
+----------+--------------------+-------+-------+--------------------+-----------+--------------+--------------------+-------------------+--------------+
|0981850006|Patio_Lawn_and_Garde| [6, 7]|    5.0|This wa

In [6]:
# casefolding
df = df.withColumn("reviewText", lower(col("reviewText")))
df.show(5)

+----------+--------------------+-------+-------+--------------------+-----------+--------------+--------------------+-------------------+--------------+
|      asin|            category|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|            summary|unixReviewTime|
+----------+--------------------+-------+-------+--------------------+-----------+--------------+--------------------+-------------------+--------------+
|0981850006|Patio_Lawn_and_Garde| [6, 7]|    5.0|this was a gift f...| 12 3, 2009|A2VNYWOPJ13AFP|Amazon Customer "...|             Delish|    1259798400|
|B00002N66D|Patio_Lawn_and_Garde| [1, 1]|    5.0|this is a very ni...| 12 3, 2012|A2E5XXXC07AGA7|               James|      Nice spreader|    1354492800|
|B00002N67U|Patio_Lawn_and_Garde| [0, 1]|    1.0|the metal base wi...|08 13, 2008|A16PX63WZIEQ13|             Finaldx|Terrible spike base|    1218585600|
|B00002N6AN|Patio_Lawn_and_Garde| [0, 0]|    4.0|for the most part...| 10 1,

In [7]:
train_df, test_df, validation_df = df.randomSplit([0.7, 0.15, 0.15], seed=42)


# Show the count of each set
print("Training set count:", train_df.count())
print("Test set count:", test_df.count())
print("Validation set count:", validation_df.count())
# Prepare test documents, which are unlabeled (id, text) tuples.
# test = 

Training set count: 55401


Test set count: 11621


Validation set count: 11807


In [ ]:
# convert category to numeric
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")

# 1. tokenize
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern=r'\s+|\t+|\d+|[(){}.!?,;:+=-_"\`~#@&*%€$§\\/]+', gaps=True)

# 3. stopwords removal
stopwords_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered_words")

## 4. tf-idf calculation with CountVectorizer
# vectorizer = CountVectorizer(inputCol=stopwords_remover.getOutputCol(), outputCol="rawFeatures", vocabSize=20000)
# idf = IDF(inputCol=vectorizer.getOutputCol(), outputCol="features")

# 4. TF-IDF calculation with HashingTF
hashingTF = HashingTF(inputCol=stopwords_remover.getOutputCol(), outputCol="rawFeatures", numFeatures=20)


# Chi-square
selector = ChiSqSelector(numTopFeatures=2000, featuresCol=hashingTF.getOutputCol(),
                         outputCol="selectedFeatures", labelCol=indexer.getOutputCol())


######
# Part 3 starts here: 
######



# Normalize in L^2
# normalizer = Normalizer(p=2.0)
# data1 = labels.zip(normalizer.transform(features))


# SVM Classifier
# svm = LinearSVC(featuresCol="selectedFeatures", labelCol=indexer.getOutputCol(), maxIter=10)
svm = LinearSVC(maxIter=10, regParam=0.1)
ovr = OneVsRest(classifier=svm, featuresCol="selectedFeatures", labelCol=indexer.getOutputCol())


# Pipeline
# pipeline = Pipeline(stages=[indexer, tokenizer, stopwords_remover, hashingTF, selector, normalizer, svm])
pipeline = Pipeline(stages=[indexer, tokenizer, stopwords_remover, hashingTF, selector, ovr])

# # ParamGrid for Cross Validation
# paramGrid = (ParamGridBuilder()
#              .addGrid(selector.numTopFeatures, [2000, 500])  # number of features selected
#              .addGrid(svm.regParam, [0.01, 0.1, 1.0])  # Regularization parameter
#              .addGrid(svm.maxIter, [10, 50])  # Maximum number of iterations
#              .build())

# 

# Fit the pipeline to training documents.
model = pipeline.fit(train_df)



# Make predictions on test documents and print columns of interest.
prediction = model.transform(test_df)
selected = prediction.select("asin", "reviewText", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print(
        "(%d, %s) -->  prediction=%f" % (
            rid, text, str(prob), prediction   # type: ignore
        )
    )